# Global Entities - Map Visualizer

#### Version 1.0

### TODO

- Plotly basic map plot
- Plotly with group level filter
- Plotly with entity filter for group
- Plotly lineplot/scatterplot/heatmap for selected entity
- Plotly Dash app

### Observations

## Setup

### Import Packages

In [20]:
from datetime import datetime, date
from pprint import pprint
from collections import OrderedDict
import os
from urllib.request import urlopen
import json

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go


# Set ipython's max row display
# pd.set_option('display.max_row', 1000)
# Set iPython's max column width
pd.set_option('display.max_columns', 50)

sns.set_style("darkgrid")
plt.rcParams['figure.figsize'] = [20, 10]

### Functions

In [21]:
def masked_heatmap():
    mask = np.zeros_like(corr)
    mask[np.triu_indices_from(mask)] = True
    fig,ax = plt.subplots()
    sns.heatmap(corr,center = 0, fmt = ".3f", square = True, annot= True, mask=mask)

## Global Variables

#### Dataset Parameters

In [22]:
file_name = '../data/silver_tables/yearly_values_per_entity.csv'

year_range = [1960, 2022]
metric_agg = 'last_year'
metric_agg = 'growth_rate'
metric_agg = '?'

In [23]:
country_list = ['United Arab Emirates', 'Japan', 'Ghana'] #For country level analysis

In [24]:
key_columns = [
    'Annual CO2 emissions', 'Annual change in primary energy consumption (%)',
    'Per capita electricity (kWh)',
    'Fossil fuels per capita (kWh)', 'Fossil fuels (% equivalent primary energy)',
    'Renewables per capita (kWh - equivalent)', 'Renewables (% electricity)',
    'Solar Generation - TWh', 'prod of Electricity from solar (TWh)',
]

corr_columns = [
    'Annual CO2 emissions', 'Annual change in primary energy consumption (%)',
    'Per capita electricity (kWh)',
    'Fossil fuels per capita (kWh)',
    'Renewables per capita (kWh - equivalent)',
]

## Data Prep

In [25]:
df_original = pd.read_csv(file_name)

In [83]:
df_original.shape

(17013, 38)

In [91]:
df_original["Land area (sq. km)"].value_counts(dropna=False)

NaN            6454
460.000         118
180.000          89
750.000          61
720.000          61
               ... 
1128570.858       1
306210.000        1
306220.000        1
306230.000        1
653150.000        1
Name: Land area (sq. km), Length: 699, dtype: int64

In [92]:
df_original["Annual CO2 emissions"].value_counts(dropna=False)

NaN            3290
47632.0          86
10992.0          85
29312.0          78
65952.0          70
               ... 
945497986.0       1
968917486.0       1
960102238.0       1
951421993.0       1
10531342.0        1
Name: Annual CO2 emissions, Length: 10778, dtype: int64

In [93]:
df_original["Income group"].value_counts(dropna=False)

NaN                    6606
High income            3629
Upper middle income    2857
Lower middle income    2570
Low income             1351
Name: Income group, dtype: int64

In [143]:
df_original["Entity_Category"].unique()

array(['Country', 'Continent', 'Bp?', 'Eia?', 'Exlusive Regions',
       'Ember?', 'Income Level', 'Other', 'World'], dtype=object)

In [154]:
63 + 50 + 43 + 23 + 50

229

In [161]:
df_original[df_original["Income group"].isnull()].query("Year == 2020 and Entity_Category == 'Country'")

,Entity,Year,Code,Region,Income group,Lending category,Fossil fuels (% equivalent primary energy),Fossil fuels (% growth),Gas Production - TWh,Land area (sq. km),Unnamed: 0.1.1.1,Coal Consumption - TWh,Renewables per capita (kWh - equivalent),Wind Generation - TWh,prod of Other renewables including bioenergy (TWh),Fossil fuels per capita (kWh),Fossil fuels (TWh),Oil Production - TWh,Coal Production - TWh,Hydro Generation - TWh,Unnamed: 0.1.1,prod of Electricity from solar (TWh),prod of Electricity from hydro (TWh),Unnamed: 0.1,"Gas (TWh, direct energy)",Annual change in primary energy consumption (%),Geo Biomass Other - TWh,"Coal (TWh, direct energy)",Per capita electricity (kWh),Oil Consumption - TWh,Renewables (% electricity),Population density (people per sq. km of land area),Solar Generation - TWh,Gas Consumption - TWh,"Oil (TWh, direct energy)",Annual CO2 emissions,prod of Electricity from wind (TWh),Entity_Category,income_factor,prioritization_score
634,Anguilla,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.234260e+05,NaN,Country,1.0,NaN
2217,Bahamas,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12899.0,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,12899.0,0.000000,0.000000,12899.0,NaN,NaN,NaN,NaN,4895.713867,NaN,0.000000,NaN,NaN,NaN,NaN,2.337684e+06,0.000000,Country,1.0,NaN
3271,Bonaire Sint Eustatius And Saba,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.006540e+05,NaN,Country,1.0,NaN
3714,Brunei,2020,NaN,NaN,NaN,NaN,NaN,NaN,126.407898,NaN,12465.0,NaN,NaN,NaN,0.000000,NaN,NaN,62.556503,NaN,NaN,12465.0,0.000000,0.000000,12465.0,NaN,NaN,NaN,NaN,9960.700195,NaN,0.000000,NaN,NaN,NaN,NaN,1.015849e+07,0.000000,Country,1.0,NaN
4493,Cape Verde,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12914.0,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,12914.0,0.010000,0.000000,12914.0,NaN,NaN,NaN,NaN,772.338623,NaN,20.000002,NaN,NaN,NaN,NaN,5.499300e+05,0.080000,Country,1.0,NaN
5241,Congo,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12485.0,NaN,NaN,NaN,0.050000,NaN,NaN,183.236160,NaN,NaN,12485.0,0.000000,1.070000,12485.0,NaN,NaN,NaN,NaN,626.075989,NaN,31.372549,NaN,NaN,NaN,NaN,3.116760e+06,0.000000,Country,1.0,NaN
5294,Cook Islands,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12918.0,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,12918.0,0.010000,0.000000,12918.0,NaN,NaN,NaN,NaN,2346.041016,NaN,25.000000,NaN,NaN,NaN,NaN,7.942300e+04,0.000000,Country,1.0,NaN
5429,Cote D'Ivoire,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,318000.00,12921.0,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,12921.0,0.020000,2.310000,12921.0,NaN,NaN,NaN,NaN,355.813568,NaN,24.423479,82.950550,NaN,NaN,NaN,1.007073e+07,0.000000,Country,1.0,31.668972
5729,Curacao,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,444.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,348.979730,NaN,NaN,NaN,3.335066e+06,NaN,Country,1.0,7511.409910
5962,Czechia,2020,NaN,NaN,NaN,NaN,75.204254,-9.698600,NaN,77198.53,1209.0,145.366714,2.762247,0.699100,5.091700,31046.357422,332.474884,NaN,118.740425,2.142300,1209.0,2.233100,2.142300,1209.0,NaN,-7.485271,5.091700,NaN,7733.166504,102.484428,12.483422,138.575929,2.233100,84.623749,NaN,8.797471e+07,0.699100,Country,1.0,1139.590430


In [152]:
df_original.query("Year == 2020 and Entity_Category == 'Country'")["Income group"].value_counts(dropna=False)

High income            63
Upper middle income    50
NaN                    50
Lower middle income    43
Low income             23
Name: Income group, dtype: int64

In [86]:
df_original["Income group"].unique()

# Land area (sq. km)## 
# Annual CO2 emissions
## Income group

# nan,    -----------------> x0
# "High income" -----------> x1
# "Upper middle income", --> x1.5
# "Lower middle income", --> x1.75
# "Low income",  ----------> x2



# Score = (Annual CO2 emissions * Land Area) * Income Factor

array(['Low income', nan, 'Upper middle income', 'Lower middle income',
       'High income'], dtype=object)

In [98]:
def income_group_mapping(text):
    if text == "High income":
        return 1.25
    if text == "Upper middle income":
        return 1.5
    if text == "Lower middle income":
        return 1.75
    if text == "Low income":
        return 2
    return 1

In [99]:
df_original["income_factor"] = df_original["Income group"].apply(lambda x: income_group_mapping(x))

In [102]:
df_original["prioritization_score"] = df_original["Annual CO2 emissions"] * df_original["Land area (sq. km)"] * df_original["income_factor"]

In [105]:
(df_original["prioritization_score"].value_counts(dropna=False))

NaN           6644
549.600000      15
49.616667       14
366.400000      14
47.496296       12
              ... 
30.675207        1
31.674788        1
26.767749        1
27.703341        1
47.640813        1
Name: prioritization_score, Length: 9721, dtype: int64

In [166]:
df_original[["Entity", "Code"]].drop_duplicates()

,Entity,Code
12,Afghanistan,AFG
149,Africa,NaN
210,Africa (Bp),NaN
295,Albania,ALB
400,Algeria,DZA
...,...,...
27626,World,WLD
27698,Yemen,NaN
27759,Yugoslavia,NaN
27781,Zambia,ZMB


In [100]:
df_original["income_factor"].value_counts(dropna=False)

1.00    6606
1.25    3629
1.50    2857
1.75    2570
2.00    1351
Name: income_factor, dtype: int64

In [85]:
df_original.query("Year == 2020")

## Land area (sq. km)
## Annual CO2 emissions
## Income group

,Entity,Year,Code,Region,Income group,Lending category,Fossil fuels (% equivalent primary energy),Fossil fuels (% growth),Gas Production - TWh,Land area (sq. km),Unnamed: 0.1.1.1,Coal Consumption - TWh,Renewables per capita (kWh - equivalent),Wind Generation - TWh,prod of Other renewables including bioenergy (TWh),Fossil fuels per capita (kWh),Fossil fuels (TWh),Oil Production - TWh,Coal Production - TWh,Hydro Generation - TWh,Unnamed: 0.1.1,prod of Electricity from solar (TWh),prod of Electricity from hydro (TWh),Unnamed: 0.1,"Gas (TWh, direct energy)",Annual change in primary energy consumption (%),Geo Biomass Other - TWh,"Coal (TWh, direct energy)",Per capita electricity (kWh),Oil Consumption - TWh,Renewables (% electricity),Population density (people per sq. km of land area),Solar Generation - TWh,Gas Consumption - TWh,"Oil (TWh, direct energy)",Annual CO2 emissions,prod of Electricity from wind (TWh),Entity_Category
71,Afghanistan,2020,AFG,South Asia,Low income,IDA,NaN,NaN,NaN,6.522300e+05,12892.0,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,12892.0,0.040000,0.620000,12892.0,NaN,NaN,NaN,NaN,20.014248,NaN,84.615395,59.684990,NaN,NaN,NaN,1.216029e+07,0.000000,Country
208,Africa,2020,NaN,NaN,NaN,NaN,89.685829,-6.705356,2312.026855,NaN,54.0,1157.901123,0.376615,21.302896,8.066416,3509.572021,4704.925781,3846.310547,1723.903687,146.373077,54.0,16.058805,146.373077,54.0,NaN,-5.539828,8.066416,NaN,628.423706,2011.052612,22.430740,NaN,16.058805,1535.972412,NaN,1.326044e+09,21.302896,Continent
264,Africa (Bp),2020,NaN,NaN,NaN,NaN,89.685829,-6.705356,2312.026855,NaN,110.0,1157.901123,NaN,21.302898,8.066416,NaN,4704.925781,3846.310547,1723.903687,146.373077,110.0,16.058805,146.373077,110.0,NaN,-5.539828,8.066416,NaN,NaN,2011.052612,22.430740,NaN,16.058805,1535.972412,NaN,NaN,21.302898,Bp?
354,Albania,2020,ALB,Europe & Central Asia,Upper middle income,IBRD,NaN,NaN,NaN,2.740000e+04,12893.0,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,12893.0,0.030000,5.280000,12893.0,NaN,NaN,NaN,NaN,1852.207153,NaN,100.000000,103.571131,NaN,NaN,NaN,4.534673e+06,0.000000,Country
459,Algeria,2020,DZA,Middle East & North Africa,Lower middle income,IBRD,99.702103,-5.827147,814.563721,2.381741e+06,166.0,4.816350,0.044457,0.007700,0.000000,14879.098633,652.463989,668.911072,NaN,0.049600,166.0,0.685000,0.049600,166.0,NaN,-5.824166,0.000000,NaN,1821.686035,211.635162,0.937777,18.411340,0.685000,436.012512,NaN,1.549955e+08,0.007700,Country
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27414,Western Sahara,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13179.0,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,13179.0,0.000000,0.000000,13179.0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,Country
27685,World,2020,WLD,NaN,NaN,NaN,82.211304,-5.385285,38615.152344,1.299798e+08,6032.0,41963.964844,2.704402,1596.428223,703.916443,16523.912109,128800.398438,48508.015625,44068.472656,4345.990234,6032.0,846.229370,4345.990234,6032.0,38455.726562,-3.985941,703.916443,41963.964844,3429.322021,48380.710938,27.864630,59.731828,846.229370,38455.726562,48380.710938,3.480726e+10,1596.428223,World
27757,Yemen,2020,NaN,NaN,NaN,NaN,NaN,NaN,3.064353,NaN,12870.0,NaN,NaN,NaN,0.000000,NaN,NaN,43.814594,NaN,NaN,12870.0,0.500000,0.000000,12870.0,NaN,NaN,NaN,NaN,100.668922,NaN,15.384615,NaN,NaN,NaN,NaN,9.768313e+06,0.000000,Country
27840,Zambia,2020,ZMB,Sub-Saharan Africa,Low income,IDA,NaN,NaN,NaN,7.433900e+05,13180.0,NaN,NaN,NaN,0.160000,NaN,NaN,NaN,NaN,NaN,13180.0,0.150000,13.670000,13180.0,NaN,NaN,NaN,NaN,844.264526,NaN,87.484352,24.729894,NaN,NaN,NaN,6.572938e+06,0.000000,Country


In [26]:
df_original = df_original.loc[
    (df_original['Year'] > year_range[0])
    & (df_original['Year'] < year_range[1])
]

In [27]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17013 entries, 12 to 27959
Data columns (total 38 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Entity                                               17013 non-null  object 
 1   Year                                                 17013 non-null  int64  
 2   Code                                                 10529 non-null  object 
 3   Region                                               10407 non-null  object 
 4   Income group                                         10407 non-null  object 
 5   Lending category                                     7279 non-null   object 
 6   Fossil fuels (% equivalent primary energy)           6068 non-null   float64
 7   Fossil fuels (% growth)                              6034 non-null   float64
 8   Gas Production - TWh                                 4053 non-nul

,Entity,Year,Code,Region,Income group,Lending category,Fossil fuels (% equivalent primary energy),Fossil fuels (% growth),Gas Production - TWh,Land area (sq. km),Unnamed: 0.1.1.1,Coal Consumption - TWh,Renewables per capita (kWh - equivalent),Wind Generation - TWh,prod of Other renewables including bioenergy (TWh),Fossil fuels per capita (kWh),Fossil fuels (TWh),Oil Production - TWh,Coal Production - TWh,Hydro Generation - TWh,Unnamed: 0.1.1,prod of Electricity from solar (TWh),prod of Electricity from hydro (TWh),Unnamed: 0.1,"Gas (TWh, direct energy)",Annual change in primary energy consumption (%),Geo Biomass Other - TWh,"Coal (TWh, direct energy)",Per capita electricity (kWh),Oil Consumption - TWh,Renewables (% electricity),Population density (people per sq. km of land area),Solar Generation - TWh,Gas Consumption - TWh,"Oil (TWh, direct energy)",Annual CO2 emissions,prod of Electricity from wind (TWh),Entity_Category
12,Afghanistan,1961,AFG,South Asia,Low income,IDA,NaN,NaN,NaN,652230.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.058547,NaN,NaN,NaN,490798.0,NaN,Country
13,Afghanistan,1962,AFG,South Asia,Low income,IDA,NaN,NaN,NaN,652230.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.337645,NaN,NaN,NaN,688594.0,NaN,Country
14,Afghanistan,1963,AFG,South Asia,Low income,IDA,NaN,NaN,NaN,652230.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.631648,NaN,NaN,NaN,706736.0,NaN,Country
15,Afghanistan,1964,AFG,South Asia,Low income,IDA,NaN,NaN,NaN,652230.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.940699,NaN,NaN,NaN,838551.0,NaN,Country
16,Afghanistan,1965,AFG,South Asia,Low income,IDA,NaN,NaN,NaN,652230.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.265041,NaN,NaN,NaN,1006917.0,NaN,Country


In [59]:
df_original['Entity_Category'].value_counts()

Country             13243
Bp?                  1549
Eia?                 1040
Exlusive Regions      361
Continent             344
Income Level          244
Ember?                110
Other                  61
World                  61
Name: Entity_Category, dtype: int64

In [106]:
category_filter = "Country"

df = df_original.copy().loc[df_original['Entity_Category'] == category_filter]

df = df.loc[df['Year'] == 2020]

In [78]:
df.head()

,Entity,Year,Code,Region,Income group,Lending category,Fossil fuels (% equivalent primary energy),Fossil fuels (% growth),Gas Production - TWh,Land area (sq. km),Unnamed: 0.1.1.1,Coal Consumption - TWh,Renewables per capita (kWh - equivalent),Wind Generation - TWh,prod of Other renewables including bioenergy (TWh),Fossil fuels per capita (kWh),Fossil fuels (TWh),Oil Production - TWh,Coal Production - TWh,Hydro Generation - TWh,Unnamed: 0.1.1,prod of Electricity from solar (TWh),prod of Electricity from hydro (TWh),Unnamed: 0.1,"Gas (TWh, direct energy)",Annual change in primary energy consumption (%),Geo Biomass Other - TWh,"Coal (TWh, direct energy)",Per capita electricity (kWh),Oil Consumption - TWh,Renewables (% electricity),Population density (people per sq. km of land area),Solar Generation - TWh,Gas Consumption - TWh,"Oil (TWh, direct energy)",Annual CO2 emissions,prod of Electricity from wind (TWh),Entity_Category
66,Afghanistan,2015,AFG,South Asia,Low income,IDA,NaN,NaN,NaN,652230.0,6068.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,6068.0,0.030,1.000,6068.0,NaN,10.476768,NaN,NaN,34.959339,NaN,87.288139,52.762987,NaN,NaN,NaN,7904133.0,0.000,Country
349,Albania,2015,ALB,Europe & Central Asia,Upper middle income,IBRD,NaN,NaN,NaN,27400.0,6107.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,6107.0,0.000,5.890,6107.0,NaN,0.069606,NaN,NaN,2043.377808,NaN,100.000000,105.135146,NaN,NaN,NaN,4495728.0,0.000,Country
454,Algeria,2015,DZA,Middle East & North Africa,Lower middle income,IBRD,99.903191,5.469954,814.111389,2381741.0,161.0,1.54679,0.015048,0.019,0.0,15528.59375,616.920288,781.826721,NaN,0.145,161.0,0.058,0.145,161.0,NaN,5.448103,0.0,NaN,1739.820923,236.148483,0.322684,16.680244,0.058,379.225006,NaN,149682942.0,0.019,Country
495,American Samoa,2015,ASM,East Asia & Pacific,Upper middle income,NaN,NaN,NaN,NaN,200.0,6146.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,6146.0,0.000,0.000,6146.0,NaN,-0.000042,NaN,NaN,3113.385498,NaN,0.000000,279.030000,NaN,NaN,NaN,NaN,0.000,Country
526,Andorra,2015,AND,Europe & Central Asia,High income,NaN,NaN,NaN,NaN,470.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,165.942553,NaN,NaN,NaN,465328.0,NaN,Country


In [79]:
df_original.describe()

,Year,Fossil fuels (% equivalent primary energy),Fossil fuels (% growth),Gas Production - TWh,Land area (sq. km),Unnamed: 0.1.1.1,Coal Consumption - TWh,Renewables per capita (kWh - equivalent),Wind Generation - TWh,prod of Other renewables including bioenergy (TWh),Fossil fuels per capita (kWh),Fossil fuels (TWh),Oil Production - TWh,Coal Production - TWh,Hydro Generation - TWh,Unnamed: 0.1.1,prod of Electricity from solar (TWh),prod of Electricity from hydro (TWh),Unnamed: 0.1,"Gas (TWh, direct energy)",Annual change in primary energy consumption (%),Geo Biomass Other - TWh,"Coal (TWh, direct energy)",Per capita electricity (kWh),Oil Consumption - TWh,Renewables (% electricity),Population density (people per sq. km of land area),Solar Generation - TWh,Gas Consumption - TWh,"Oil (TWh, direct energy)",Annual CO2 emissions,prod of Electricity from wind (TWh)
count,17013.000000,6068.000000,6034.000000,4053.000000,1.055900e+04,13164.000000,6134.000000,4862.000000,5939.000000,8899.000000,4840.000000,6068.000000,4408.000000,2500.000000,6153.000000,13164.000000,8939.000000,9170.000000,13164.000000,57.000000,12261.000000,5948.000000,57.000000,6220.000000,6160.000000,7008.000000,9717.000000,5924.000000,6094.000000,57.000000,1.372300e+04,8953.000000
mean,1992.379533,85.831892,3.317557,1775.709862,1.433329e+06,6581.873215,1625.004354,4.339072,13.805802,10.535739,31653.758962,5136.950645,3466.583599,3274.685893,136.736280,6581.873215,4.423860,105.856874,6581.873215,21492.190695,4.125264,12.445990,28584.640556,3950.509462,2228.388291,30.488270,161.785499,5.047541,1228.198540,39160.740543,4.750953e+08,12.191714
std,17.069374,14.603942,22.875763,4042.829666,9.901976e+06,3800.896830,4907.816698,12.693478,81.229086,45.642700,33092.712344,13737.030117,6955.447481,6908.166717,374.128795,3800.896830,37.610140,346.000705,3800.896830,9564.826179,27.879834,45.803265,10007.255369,5026.958829,5840.259523,30.934145,475.889730,38.265463,3413.741868,8988.199100,2.184178e+09,81.726038
min,1961.000000,12.804697,-49.590828,0.000000,1.000000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,155.807388,0.097381,0.000000,0.339596,0.000000,0.000000,0.000000,0.000000,0.000000,6303.798828,-95.005081,0.000000,16060.809570,0.000000,0.097381,0.000000,0.098625,0.000000,0.000000,17989.603516,3.664000e+03,0.000000
25%,1978.000000,80.509890,-0.531910,44.498001,2.318000e+04,3290.750000,4.537538,0.208465,0.000000,0.000000,10325.921631,149.484257,100.963156,38.597149,1.389000,3290.750000,0.000000,0.100000,3290.750000,14118.032227,-0.349981,0.000000,20363.539062,596.250275,80.953899,3.552361,17.792095,0.000000,12.293094,33680.054688,6.998240e+05,0.000000
50%,1993.000000,90.139271,2.589911,192.537277,1.118900e+05,6581.500000,46.895945,0.899160,0.000000,0.020000,25337.834961,418.682419,467.141678,357.479385,10.712626,6581.500000,0.000000,3.565500,6581.500000,20265.484375,2.460420,0.201000,25963.681641,2502.079101,208.418251,18.145459,60.702633,0.000000,81.496605,38151.847656,7.081458e+06,0.000000
75%,2007.000000,96.528872,5.987856,992.275024,4.994600e+05,9872.250000,442.860115,3.228757,0.221320,1.333982,40323.159180,1808.173248,3527.187439,2806.387512,59.555000,9872.250000,0.010000,28.667310,9872.250000,29315.390625,6.231642,3.617000,40175.359375,5627.821900,999.276443,53.127329,138.575929,0.016565,429.343689,47172.609375,6.293776e+07,0.040000
max,2021.000000,100.000015,1553.104980,40368.828125,1.299870e+08,13180.000000,45161.207031,153.883406,1861.939819,762.782654,308704.218750,136131.468750,52181.949219,46550.605469,4345.990234,13180.000000,1032.501221,4345.990234,13180.000000,40374.605469,1553.104980,762.782654,45161.207031,56781.601560,53368.628906,100.000000,7965.878492,1032.501221,40374.605469,53368.628906,3.670250e+10,1861.939819


## Visualizations

In [80]:
x = 'Code'
c = 'Entity'

# y = 'Renewables per capita (kWh - equivalent)'
y = 'Fossil fuels per capita (kWh)'

In [112]:
# Define data columns for the drop-down filter
cols_dd = ["Fossil fuels (% growth)", "Land area (sq. km)", "Per capita electricity (kWh)", "prioritization_score", "income_factor"]
# cols_dd = df_original.columns

# We need to add this to select which trace is going to be visible
visible = np.array(cols_dd)

# define traces and buttons at once
traces = []
buttons = []
for value in cols_dd:
    traces.append(go.Choropleth(locations=df[x], z=df[value], text=df[c], colorbar_title=value, visible=True if value==cols_dd[0] else False))

    buttons.append(dict(label=value, method="update", args=[{"visible":list(visible==value)}, {"title":f"<b>{value}</b>"}]))

updatemenus = [{"active":0, "buttons":buttons,}]

# Show figure
fig = go.Figure(data=traces, layout=dict(updatemenus=updatemenus))

# This is in order to get the first title displayed correctly
first_title = cols_dd[0]
fig.update_layout(title=f"<b>{first_title}</b>",title_x=0.5)

# Update layout margins to enhance the plot size and display
fig.update_layout(margin={"r":0,"t":50,"l":0,"b":0})
fig.show()

In [131]:
df_2020 = df_original.query("Year == 2020").copy()
df_2020["prioritization_score_normalized"] =  df_2020["prioritization_score"] / max(df_2020["prioritization_score"]) * 100

In [132]:
df_2020[["Entity", "prioritization_score_normalized"]].sort_values("prioritization_score_normalized", ascending=False).head(10)

,Entity,prioritization_score_normalized
22308,Singapore,100.000000
2305,Bahrain,70.271380
22403,Sint Maarten (Dutch Part),28.933892
3032,Bermuda,18.246447
20888,Qatar,14.646601
14850,Maldives,11.338497
25100,Trinidad And Tobago,10.921994
5729,Curacao,9.481722
13184,Kuwait,7.874830
14983,Malta,7.864300


In [134]:
df_2020.query("income_factor == 1.25")[["Entity", "prioritization_score_normalized"]].sort_values("prioritization_score_normalized", ascending=False).head(10)

,Entity,prioritization_score_normalized
22308,Singapore,100.000000
2305,Bahrain,70.271380
22403,Sint Maarten (Dutch Part),28.933892
3032,Bermuda,18.246447
20888,Qatar,14.646601
25100,Trinidad And Tobago,10.921994
13184,Kuwait,7.874830
14983,Malta,7.864300
1136,Aruba,6.602737
16664,Netherlands,6.471822


In [125]:
df_original.query("Year == 2020")[["Entity", "prioritization_score"]].sort_values("prioritization_score", ascending=False).head(10)

,Entity,prioritization_score
22308,Singapore,79219.888579
2305,Bahrain,55668.909236
22403,Sint Maarten (Dutch Part),22921.397059
3032,Bermuda,14454.814815
20888,Qatar,11603.020779
14850,Maldives,8982.345000
25100,Trinidad And Tobago,8652.391326
5729,Curacao,7511.409910
13184,Kuwait,6238.431327
14983,Malta,6230.089844
